# L02 Model creation and fine tunning

**Siamese neural networks**

Tell how similar two images are by comparing them in a model.Feed neural network with two images and then create an encoding of each images. The final result will be two vectors that will be compared.


## 1. Import libraries

In [2]:
import torch
import torch.nn as nn


## 2. Create models

### 2.1 Siamese Network

In [9]:
class SiameseNeuralNetwork(nn.Module):
    """
    Siamese Neural Network for performing image verification. Consists on 3 convolutional layers
    and 3 fully connected layers. Input and output shapes can be tune up.

    Parameters:
    number_channels (int): Number of channels of the input image.
    length_a (int): Number of channels for first convolutional layer.
    length_b (int): Number of channels for second convolutional layer.
    length_c (int): Number of channels for third convolutional layer.
    flatten_a (int): Number of neurons for the first fully connected layer.
    flatten_b (int): Number of neurons for the second fully connected layer.

    Method:
    forward_all(x): Computes feature vector for one sample.
    forward(input_a, input_b): Computes feature vector for two samples.
    """

    def __init__(self,number_channels, length_a, length_b, length_c, flatten_a, flatten_b):
        super(SiameseNeuralNetwork, self).__init__(number_channels)
        self.number_channels = number_channels

        # Create three convolutional layers
        self.cnn1 = nn.Sequential(
            nn.Conv2d(number_channels, length_a, kernel_size = 11, stride = 4),
            nn.ReLU(implace=True),
            nn.MaxPool2d(3, stride=2),
    
            nn.Conv2d(length_a, length_b, kernel_size = 5, stride = 1),
            nn.ReLU(implace=True),
            nn.MaxPool2d(2, stride=2),
        
            nn.Conv2d(length_b, length_c,  kernel_size = 3, stride = 1),
            nn.ReLU(implace=True),
        )

        # Create two fully connected layers
        self.fc1 = nn.Sequential(
            nn.Linear(length_c, flatten_a),
            nn.ReLU(inplace=True),
            nn.Linear(flatten_a, flatten_b),
            nn.ReLU(inplace=True),
            nn.Linear(flatten_b, 2)
        )
    
    def forward_all(self, x):
        """
        Pass image tensor through convolutional and fully connected layer.

        Parameters:
        x (tensor) -> image tensor from dataset.

        Output:
        output (array) -> encoded vector.
        """
        output = self.cnn1(x)
        # Take batch dimension and multiply the rest.
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input_a, input_b):
        """
        Main forward function that passes two images in the Siamese Network
        for computing encoding vectors.

        Parameters:
        input_a (tensor) -> image tensor from dataset.
        input_b (tensor) -> image tensor from dataset.

        Output:
        output1 (array) -> encoded vector.
        output2 (array) -> encoded vector.
        """
        output1 = self.forward_all(input_a)
        output2 = self.forward_all(input_b)

        return output1, output2